In [ ]:
import numpy as np
import sklearn
import scipy.optimize
import pandas as pd
import math
import random
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import json
import random
import gzip
import matplotlib.pyplot as plt

In [ ]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

allRatings = []
userRatings = defaultdict(list)
users = []
items = []
category = []
for l in readGz("train.json.gz"):
    user = l['reviewerID']
    allRatings.append(l['rating'])
    userRatings[user].append(l['rating'])
    users.append(l['reviewerID'])
    items.append(l['itemID'])
    category.append(l['categories'])

    
users_train, users_val = users[:100000], users[100000:]
items_train, items_val  = items[:100000], items[100000:]
category_train, category_val = category[:100000], category[100000:]
ratings_train, ratings_val = allRatings[:100000], allRatings[100000:]

In [3]:
purchasers_dict = defaultdict(list)
for u,i in zip(users, items):
    purchasers_dict[u].append(i)



unique_users = list(set(users))
unique_items = list(set(items))

non_purchasers = []
unpurchased_items = []
while len(non_purchasers) <= 99999:
    u = random.choice(unique_users)
    i = random.choice(unique_items)
    if i not in purchasers_dict[u]:
        non_purchasers.append(u)
        unpurchased_items.append(i)
        
complete_users_val = users_val + non_purchasers
complete_items_val = items_val + unpurchased_items
actual_purchase_val = [1]*100000 + [0]*100000

# Purhcase Prediction 
## Model 1 - Overlapping categories

In [ ]:
user_category_dict = defaultdict(list)
item_category_dict = defaultdict(list)

for u,c in zip(users_train, category_train):
    for sublist in c:
        for subcat in sublist:
            if subcat in user_category_dict[u]: continue
            user_category_dict[u].append(subcat)

for i,c in zip(items_train,category_train):
    for sublist in c:
        for subcat in sublist:
            if subcat in item_category_dict[i]: continue
            item_category_dict[i].append(subcat)
            
pred_val = []

common_categories = [1,2,3,4,5,6,7,8]
accuracy = []

for k in common_categories:
    pred_val = []
    accuracy_val = 0
    for u,i in zip(complete_users_val, complete_items_val):
        common = 0
        c = item_category_dict[i]
        if len(c) == 0:
            pred_val.append(0)
            continue
        for subcat in c:
            if subcat in user_category_dict[u]:
                common += 1
        if common >= k:
            pred_val.append(1)
        else:
            pred_val.append(0)
  
    accuracy_val = float(sum([(a == b) for a,b in zip(pred_val, actual_purchase_val)]))/len(actual_purchase_val)  
    accuracy.append(accuracy_val)

In [ ]:
k = common_categories[accuracy.index(max(accuracy))]

pred_val = []
for u,i in zip(complete_users_val, complete_items_val):
    common = 0
    c = item_category_dict[i]
    if len(c) == 0:
        pred_val.append(0)
        continue
    for subcat in c:
        if subcat in user_category_dict[u]:
            common += 1
    if common >= k:
        pred_val.append(1)
    else:
        pred_val.append(0)
        
accuracy_val = float(sum([(a == b) for a,b in zip(pred_val, actual_purchase_val)]))/len(actual_purchase_val)  

In [ ]:
print "Common categories = " + str(k) 
print "Accuracy = " + str(accuracy_val)

In [ ]:
cm = confusion_matrix(pred_val, actual_purchase_val)
print cm

In [ ]:
user_category_dict = defaultdict(list)
item_category_dict = defaultdict(list)

for u,c in zip(users, category):
    for sublist in c:
        for subcat in sublist:
            if subcat in user_category_dict[u]: continue
            user_category_dict[u].append(subcat)
            
for i,c in zip(items, category):
    for sublist in c:
        for subcat in sublist:
            if subcat in item_category_dict[i]: continue
            item_category_dict[i].append(subcat)
            
            
predictions = open("predictions_Purchase.csv", 'w')

for l in open("pairs_Purchase.txt"):
    common = 0
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    c = item_category_dict[i]
    if len(c) == 0:
        predictions.write(u + '-' + i + ",0\n")
        continue
    for subcat in c:
        if subcat in user_category_dict[u]:
            common += 1
    if common >= 3: 
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")

predictions.close()

In [ ]:
########  EDA #####################

all_item_cat = defaultdict(list)

for i,c in zip(items,category):
    for sublist in c:
        for subcat in sublist:
            if subcat in all_item_cat[i]: continue
            all_item_cat[i].append(subcat)
            
category_list = []

for i in items:
    category_list.append(all_item_cat[i])
    
catCount = defaultdict(int)

for u,c in zip(users, category_list):
    for subcat in c:
        catCount[subcat] += 1
    
mostPopularCat = [(catCount[x], x) for x in catCount]
mostPopularCat.sort(reverse = True)
mostPopularCat = pd.DataFrame(np.array(mostPopularCat), columns = ('Freq','Category'))
mostPopularCat['Freq'] = mostPopularCat['Freq'].astype(int)

In [ ]:
mostPopularCat[:10]
plt.bar(mostPopularCat['Category'][:10], mostPopularCat['Freq'][:10])

## Model 2 - Similarity between items

In [ ]:
########### Finding similarity between items purchased by users, the similarity is > than a threshold then 1 else 0 ###############

user_category_dict = defaultdict(list)
item_category_dict = defaultdict(list)

for u,i,c in zip(users_train, items_train, category_train):
    for sublist in c:
        for subcat in sublist:
            if subcat in user_category_dict[u]: continue
            user_category_dict[u].append(subcat)
            if subcat in item_category_dict[i]: continue
            item_category_dict[i].append(subcat)
            

threshold = list(np.arange(0.18,0.19,0.001))


def jaccard_similarity(set1,set2):
    common = set1.intersection(set2)
    return float(len(common)) / (len(set1) + len(set2) - len(common))


for k in threshold:
    pred_val = []
    for u,i in zip(complete_users_val,complete_items_val):
        new_c = set(item_category_dict[i])
        user_c = set(user_category_dict[u])
        if len(user_c) <> 0:
            similarity = jaccard_similarity(new_c,user_c)
            if similarity >= k:
                pred_val.append(1)
            else:
                pred_val.append(0)
        else:
            pred_val.append(1)
    accuracy_val = float(sum([(a == b) for a,b in zip(pred_val, actual_purchase_val)]))/len(actual_purchase_val)  
    print k,accuracy_val

In [ ]:
user_category_dict = defaultdict(list)
item_category_dict = defaultdict(list)

for u,i,c in zip(users, items, category):
    for sublist in c:
        for subcat in sublist:
            if subcat not in user_category_dict[u]:
                user_category_dict[u].append(subcat)
            if subcat not in item_category_dict[i]: 
                item_category_dict[i].append(subcat)
            
predictions = open("predictions_Purchase.csv", 'w')

for l in open("pairs_Purchase.txt"):
    common = 0
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    new_c = set(item_category_dict[i])
    user_c = set(user_category_dict[u])
    if len(user_c) <> 0:
        similarity = jaccard_similarity(new_c,user_c)
        if similarity >= 0.182:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
    else:
        predictions.write(u + '-' + i + ",1\n")

predictions.close()

## Model 3 - Combining similarity and most popular item concept

In [ ]:
itemCount = defaultdict(int)
totalPurchases = 0

for user, item in zip(users, items):
    itemCount[item] += 1
    totalPurchases += 1

mostPopular = [(itemCount[x], x) for x in itemCount]
mostPopular.sort()
mostPopular.reverse()

user_category_dict = defaultdict(list)
item_category_dict = defaultdict(list)

for u,i,c in zip(users, items, category):
    for sublist in c:
        for subcat in sublist:
            if subcat not in user_category_dict[u]:
                user_category_dict[u].append(subcat)
            if subcat not in item_category_dict[i]: 
                item_category_dict[i].append(subcat)
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases*0.53: break
           
predictions = open("predictions_Purchase.csv", 'w')

for l in open("pairs_Purchase.txt"):
    common = 0
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    new_c = set(item_category_dict[i])
    user_c = set(user_category_dict[u])
    if len(user_c) <> 0:
        similarity = jaccard_similarity(new_c,user_c)
        if similarity >= 0.182 or i in return1:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
    elif len(user_c) == 0 and i in return1:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")

predictions.close()

# Rating Prediction

## Model 1 - Collaborative filtering

In [ ]:
#### Rating prediction ##########3

userRating_train = defaultdict(list)
userRatings = defaultdict(list)
itemRatings = defaultdict(list)
itemCategory_train = defaultdict(list)
userItem_train = defaultdict(list)

for u,i,r,c in zip(users_train, items_train, ratings_train, category_train):
    userRating_train[u].append((i,r))
    userItem_train[u].append(i)
    userRatings[u].append(r)
    itemRatings[i].append(r)
    for sublist in c:
        for subcat in sublist:
            if subcat not in itemCategory_train[i]:
                itemCategory_train[i].append(subcat)
                
globalAvg = np.mean(ratings_train)
userAverage = {}
for u in userRatings:
      userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
        
itemAverage = {}
for i in itemRatings:
      itemAverage[i] = sum(itemRatings[i]) / len(itemRatings[i])

In [ ]:
def jaccard_similarity(set1,set2):
    common = set1.intersection(set2)
    return float(len(common)) / (len(set1) + len(set2) - len(common))

pred_rating = []

for u,i in zip(users_val, items_val):
    category_i = set(itemCategory_train[i])
    user_ratings = userRating_train[u]
    if u in userItem_train.keys() and len(category_i) <> 0:
        rating_temp = 0.0
        sim = 0
        for j,r in user_ratings:
            category_j = set(itemCategory_train[j])
            sim = jaccard_similarity(category_i,category_j)
            rating_temp += round(r*sim,4)
            sim_norm += sim
        if sim_norm <> 0:
            rating = round(rating_temp/sim_norm,4)
            if rating > 5:
                pred_rating.append(5)
            elif rating < 0:
                pred_rating.append(0)
            else:
                pred_rating.append(rating)
        else:
            pred_rating.append(round(rating_temp,4))
    elif u not in userItem_train.keys() and len(category_i) <> 0:
        pred_rating.append(itemAverage[i])
    elif u in userItem_train.keys() and len(category_i) == 0:
        pred_rating.append(userAverage[u])
    else:
        pred_rating.append(0)

In [ ]:
diff = np.array(ratings_val) - np.array(pred_rating)
MSE = sum(diff*diff.T)/len(pred_rating)
print MSE

## Model 2 - Latent Factor Model

In [77]:
def gradient_descent(lam, usersTrain, itemsTrain, ratingTrain):
    
    userRating_train, itemRating_train = defaultdict(list), defaultdict(list)

    for u,i,r in zip(usersTrain, itemsTrain, ratingTrain):
        userRating_train[u].append((i,r))
        itemRating_train[i].append((u,r))  
    
    alpha, alpha_temp, iteration = 0, 0, 0
    betaUser, betaItem =  defaultdict(float), defaultdict(float)
    betaUser_temp, betaItem_temp = defaultdict(float),defaultdict(float)

    for u in usersTrain:
        betaUser[u], betaUser_temp[u] = 0, 0
    for i in itemsTrain:
        betaItem[i], betaItem_temp[i] = 0, 0

    while iteration <= 500:
        #betaUser
        for u in betaUser.keys():
            denominator = lam + len(userRating_train[u])
            user_ratings = userRating_train[u]
            numerator = 0
            for i,r in user_ratings:
                numerator += (r - (alpha + betaItem[i]))
            betaUser_temp[u] = numerator*1.0/denominator
        #betaItem
        for i in betaItem.keys():
            denominator = lam + len(itemRating_train[i])
            item_ratings = itemRating_train[i]
            numerator = 0
            for u,r in item_ratings:
                numerator += (r - (alpha + betaUser[u]))
            betaItem_temp[i] = numerator*1.0/denominator
        #alpha
        numerator = 0
        for u,i,r in zip(usersTrain, itemsTrain, ratingTrain):
            numerator += (r - (betaUser[u] + betaItem[i]))
        alpha_temp = numerator*1.0/len(usersTrain)

        betaUser[u] = betaUser_temp[u]
        betaItem[i] = betaItem_temp[i]
        alpha = alpha_temp
        
        iteration += 1
    
    return(alpha, betaUser, betaItem)    

In [78]:
lam = [0, 0.001,0.01,0.1,1,2,3,4]
lamMSE = {}

In [80]:
for l in lam:
    alpha, betaUser, betaItem = gradient_descent(l, users_train, items_train, ratings_train)
    
    rating_pred_val = []
    for u,i in zip(users_train, items_train):
        prediction = alpha + betaUser[u] + betaItem[i]
        if prediction > 5:
            prediction = 5
        elif prediction < 0:
            prediction = 0
        rating_pred_val.append(prediction)

    diff = np.array(ratings_val) - np.array(rating_pred_val)
    MSE = sum(diff*diff.T)/len(rating_pred_val)
    
    lamMSE[l] = MSE
    
lam = min(lamMSE, key = lamMSE.get)

KeyboardInterrupt: 

In [ ]:
print lamMSE
print lam

In [44]:
lam = 3
alpha, betaUser, betaItem = gradient_descent(lam, users, items, allRatings)

TypeError: gradient_descent() takes exactly 6 arguments (4 given)

In [ ]:
predictions_ratings = open("predictions_Rating.csv", 'w')

for l in open("pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        predictions_ratings.write(l)
        continue
    u,i = l.strip().split('-')
    prediction = alpha + betaUser[u] + betaItem[i]
    if prediction > 5:
        prediction = 5
    elif prediction < 0:
        prediction = 0
    
    predictions_ratings.write(u + '-' + i + ',' + str(prediction) + '\n')

    
predictions_ratings.close()

In [11]:
1.0010002/

1.0002995004396604